# 不等式のQUBO変換を用いて基底エネルギー以下のサンプルを得る
（結果）　サンプルが選ばれすぎている。<br/>
（コメント）　MCMCが選ばれなさすぎただけかもしれない、エラーがあるか確認せよ

一方で，不等式だと，直接表せないので，例えば，$\sum_i a_i x_i  >=1$は，複数の等式，$\sum_i a_i x_i  = 1$, $\sum_i a_i x_i  = 2$, $\sum_i a_i x_i  = 3,…$で表すことにして，その等式分だけ$(\sum_i a_i x_i -1)^2$, $(\sum_i a_i x_i -2)^2$, $(\sum_i a_i x_i -3)^2$と制約項を加えるということになります．ただ，この中の等式のどれかだけ成立すればいいので，このうちどの等式が満たされるかを表すadditional bitが必要になるというカラクリです．
これでわかりますかね…


充足可能性問題（じゅうそくかのうせいもんだい、satisfiability problem, SAT）は、一つの命題論理式が与えられたとき、それに含まれる変数の値を偽 (False) あるいは真 (True) にうまく定めることによって全体の値を'真'にできるか、という問題
- [The Boolean Satisfiability Problem : Advanced Math](https://www.youtube.com/watch?v=sBTTRwpJWi0)
- [pip install python-sat](https://pysathq.github.io/)
- [数独を一瞬で解く by SATソルバー](https://qiita.com/okmt1230z/items/63f49e021c94077c343e#fn2)

In [140]:
#!pip install sklearn

In [141]:
#import
import pandas as pd
import numpy as np
#from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import SelectKBest, f_regression
#from sklearn.feature_selection import SelectPercentile,chi2
import random

In [142]:
#X : 患者ID（index）, 遺伝子名(columns), 標準化された遺伝子発現（要素）

#元のデータから、標準偏差が 低い(患者間で違いがあまりない)遺伝子は除外されている
X = pd.read_csv('../input/geneX_df.txt',sep ='\t')# 説明変数
print("X.shape is {}".format(X.shape))

X.head()

X.shape is (781, 16799)


,ARHGEF10L,HIF3A,RNF17,RNF11,REM1,RTN4RL2,C16orf13,C16orf11,FGFR1OP2,TSKS,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
TCGA-A8-A092-01,-0.92667,-4.836158,-0.545559,-0.425362,-2.511796,1.420118,-0.076931,-0.583953,-0.064954,-1.262546,...,-0.727478,-1.319172,-0.534283,-0.871063,0.137041,-0.494064,-0.396097,-1.178462,-1.748619,-0.659749
TCGA-D8-A1JK-01,-0.83967,-3.260158,0.420041,0.718538,-0.322796,-1.267982,0.252869,-0.583953,-0.031154,-1.634746,...,-0.165278,-1.691372,0.783017,0.037337,-0.051459,0.730136,1.388803,1.339138,-1.924519,0.075451
TCGA-E2-A10C-01,-0.11937,-4.646958,-0.545559,-0.453962,-1.073796,-1.506382,1.084969,1.680847,0.010146,0.890654,...,-0.166078,-1.129972,-0.350883,-0.681864,-0.206159,-0.406364,0.354103,-0.003162,-0.903719,-0.195349
TCGA-BH-A0C7-01,0.28803,-1.599358,-0.545559,-0.211162,-2.041396,-0.156182,-0.861231,-0.583953,-0.249454,-1.029746,...,-0.122478,-0.488172,-0.616583,-1.243263,0.330741,-0.188764,0.420403,-0.456262,0.914881,-0.427849
TCGA-A8-A06N-01,0.61053,-4.268858,-0.545559,0.764638,-2.182996,-1.431682,-0.876131,-0.583953,0.560146,-1.634746,...,-0.429378,2.566228,-1.073483,-1.243263,0.133141,-2.727864,-0.306397,-0.882762,-3.308419,-0.224949


In [143]:
type(X.iloc[2, 4]) #numpy.float64

numpy.float64

In [144]:
num_samples = X.shape[0]
num_cols = X.shape[1]

selected_col_num = 3

In [145]:
#Xの要素の最大値、最小値
max_X_element = X.max(axis=1).max() #20.4446452623
min_X_element = X.min(axis=1).min()  #-9.741414

In [146]:
X = X.astype(float)

In [147]:
y_df = pd.read_csv('../input/ERY_df.txt' ,sep ='\t')# 目的変数
print('type of y is {}'.format(type(y_df)))
y_df.head()

type of y is <class 'pandas.core.frame.DataFrame'>


,ER_Status_nature2012
TCGA-A8-A092-01,Positive
TCGA-D8-A1JK-01,Negative
TCGA-E2-A10C-01,Positive
TCGA-BH-A0C7-01,Positive
TCGA-A8-A06N-01,Positive


In [148]:
for yy in range(y_df.shape[0]):
    if  y_df.iloc[yy, 0] == 'Positive':
         y_df.iloc[yy, 0] = 1
    else:
        y_df.iloc[yy, 0] = 0

In [149]:
y = y_df['ER_Status_nature2012'] #to Series
y.head()

TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
Name: ER_Status_nature2012, dtype: int64

## 特徴量選択
 - select_cols, else_colsを決める

### 問題発生
- SelectKBest ...`ValueError: Input X must be non-negative.`<br/>
SelectKBestを選んだ理由➡上位〇〇個を出してくれるから
reason: Pearson's chi square test (goodness of fit) does not apply to negative values. It's logical because the chi square test assumes frequencies distribution and a frequency can't be a negative number.
[Feature selection using scikit-learn](https://stackoverrun.com/ja/q/7074850)

- SelectPercentile ...`ValueError: Input X must be non-negative.`<br/>

In [150]:
#selector = SelectKBest(score_func=chi2, k=selected_col_num) #S
selector = SelectKBest(score_func=f_regression, k=selected_col_num) 
#selector = SelectPercentile(score_func=chi2, percentile=40) 
selector.fit(X, y)
mask = selector.get_support() 

In [151]:
select_cols = []
else_cols = []
for ii in range(len(mask)):
    if mask[ii] == 0:
        else_cols.append(X.columns[ii])
    else:
         select_cols.append(X.columns[ii])

select_cols

['CA12', 'AGR3', 'ESR1']

## すべての不等式が成り立つ：条件

- 不等式付き問題（Problems with Inequalities）
- [整数重みナップザック問題](https://quantum.fixstars.com/techresouces/research/ising-model-formulation/knapsack/)
- [イジングモデルでナップサック問題を解く工夫を考えてみた](https://qiita.com/YuichiroMinato/items/4ee83e3150bf693f3bb2)

In [152]:
# W : ay - byの最大値

#xの要素で最大値とxの要素の最小値を知れば推定はできる
W = int((max_X_element *1*num_samples - min_X_element*1*num_samples)//1)

In [153]:
def make_y(y_before):
    select_index = random.randrange(num_samples)
    y_before[select_index] = 1 - y_before[select_index] 
    y_after = y_before
    return y_after

In [154]:
def make_ax_by(y_after):
    ax_by = []
    for i in range(selected_col_num):
        for j in range(num_cols - selected_col_num):
            ax_by.append(np.dot(X[select_cols[i]], y_after) - np.dot(X[else_cols[j]], y_after) +1)
    return ax_by 

In [155]:
def H(ax_by, index_ax_by):
    z = [0]*W
        
    each_ax_by = ax_by[index_ax_by]
    int_each_axby = each_ax_by //1
    sum_w_z = 0
    
    if int_each_axby >= 1:
        z[int(int_each_axby)] = 1
        sum_w_z = sum([int_each_axby*z[w] for w in range(W)])
    else:
        z[0] = 1
        sum_w_z = sum([1*z[w] for w in range(W)])
        
    sum_z = sum(z)

    ans_h =  (1-sum_z)**2 + (sum_w_z -  int(int_each_axby))**2
    return ans_h

In [108]:
tt = 0
ax_by = make_ax_by(y)
for index_ax_by in range(len(ax_by)):
    ans_h = H(ax_by,  index_ax_by)
    if ans_h != 0:
        print(tt)
        print(ans_h)
    tt += 1

705
904402.0
1201
112226.0
1203
15626.0
1840
1117250.0
1982
3276101.0
2041
547601.0
2312
795665.0
2852
283025.0
2969
30626.0
3005
32401.0
3177
106277.0
3394
50177.0
3474
525626.0
3521
1155626.0
3768
44522.0
3832
1392401.0
3988
314722.0
4438
785.0
4586
44522.0
4765
1648657.0
4850
1739762.0
4976
4363922.0
4979
498437.0
5521
157610.0
5859
165650.0
5906
44945.0
6505
26897.0
6579
60517.0
6580
541697.0
6639
239122.0
7000
889250.0
7016
431650.0
7189
3721042.0
7255
900602.0
7401
105626.0
7438
7570.0
7647
11026.0
7812
42437.0
7948
962.0
8262
44945.0
8276
158405.0
8337
1024145.0
8952
216226.0
9769
528530.0
9806
235226.0
9830
213445.0
10105
203402.0
10196
162410.0
10207
168101.0
10240
66565.0
10316
1181570.0
10654
112226.0
10721
37637.0
10787
1322501.0
10896
197.0
11017
594442.0
11020
96101.0
11045
1203410.0
11262
9678322.0
11340
474722.0
11475
6562.0
11599
2893402.0
11808
328330.0
11835
422501.0
12162
2805626.0
12547
1060901.0
12628
908210.0
12636
143642.0
12977
1951610.0
13007
14401.0
13581
217

In [126]:
each_ax_by = ax_by[705]
each_ax_by 

-950.4025273829006

In [127]:
def check_ax_by(y_after, num):
    ax_by = []
    for i in range(selected_col_num):
        for j in range(num_cols - selected_col_num):
            ax_by.append(np.dot(X[select_cols[i]], y_after) - np.dot(X[else_cols[j]], y_after) +1)
            if len(ax_by) == num:
                print(i, j )

In [128]:
check_ax_by(y, 706)

0 705


In [129]:
np.dot(X[select_cols[0]], y) - np.dot(X[else_cols[705]], y)  #なぜ負になるのかを考える

-951.4025273829006

In [156]:
def H_sum(ax_by):
        H_sum =  sum([H(ax_by, index_ax_by) for index_ax_by in range(len(ax_by))])
        return H_sum

In [157]:
#know ori_H_sum
ax_by = make_ax_by(y)
ori_H_sum = H_sum(ax_by)
ori_H_sum #70464156.0

70464156.0

In [158]:
#find ans_y which H is as low as ori_y

y_after =  make_y(y)
ax_by = make_ax_by(y_after)
each_H_sum = H_sum(ax_by)

if each_H_sum <= ori_H_sum:
    print(y_after) 
    print(each_H_sum) #70330698.0

TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    1
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    1
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    1
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0DO-

In [ ]:
#少なめloop
iii = 0
for ii in range(100):
    y_after =  make_y(y_after)
    ax_by = make_ax_by(y_after)
    a_H_sum = H_sum(ax_by)
    if a_H_sum <= ori_H_sum:
        iii += 1
        print(iii)
        print(a_H_sum)
        print(y_after)

1
70134819.0
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    1
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    1
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    1
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0


In [ ]:
#中程度loop
for ii in range(1000):
    y_after =  make_y(y_after)
    ax_by = make_ax_by(y_after)
    a_H_sum = H_sum(ax_by)

    if a_H_sum <= ori_H_sum:
        print(y_after) 

In [ ]:
#多いloop
for ii in range(2**num_samples):
    y_after =  make_y(y_after)
    ax_by = make_ax_by(y_after)
    a_H_sum = H_sum(ax_by)

    if a_H_sum <= ori_H_sum:
        print(y_after) 

## py-sat

In [ ]:
from pysat.formula import CNF
from pysat.examples.lbx import LBX

In [ ]:
formula = CNF(from_file='input.cnf')
mcsls = LBX(formula)
for mcs in mcsls.enumerate():
    print(mcs)

# d-wave 
-[What is Quantum Annealing?](https://docs.dwavesys.com/docs/latest/c_gs_2.html)

## neal usage
- [dwave-neal](https://docs.ocean.dwavesys.com/projects/neal/en/latest/)

In [ ]:
#!pip install dwave-neal

In [139]:
import neal

sampler = neal.SimulatedAnnealingSampler()

h = {0: -1, 1: -1}
J = {(0, 1): -1}
response = sampler.sample_ising(h, J)
response

SampleSet(rec.array([([1, 1], -3., 1)],
          dtype=[('sample', 'i1', (2,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1], {'beta_range': [0.34657359027997264, 4.605170185988092], 'beta_schedule_type': 'geometric'}, 'SPIN')

- [dwave-neal→Simulated Annealing Sampler](https://dwave-neal-docs.readthedocs.io/en/latest/reference/sampler.html)

In [ ]:
#classSimulatedAnnealingSampler

sampler = neal.SimulatedAnnealingSampler()

h = {'a': 0.0, 'b': 0.0, 'c': 0.0}
J = {('a', 'b'): 1.0, ('b', 'c'): 1.0, ('a', 'c'): 1.0}

response = sampler.sample_ising(h, J)

for sample in response:  # doctest: +SKIP
    print(sample)

## dimod usage
- [dimod](https://docs.ocean.dwavesys.com/projects/dimod/en/latest/)
- [Ising, QUBO, and BQMs](https://docs.ocean.dwavesys.com/projects/dimod/en/latest/reference/bqm/binary_quadratic_model.html)

$$ H(x\in \{0,1\}^3) = x_1 + x_2 + 2x_1 * x_2 - x_2 * x_3$$


- 北井さんのコードは適用できない

In [ ]:
#!pip freeze #dimod==0.9.3

In [ ]:
import dimod

In [ ]:
#lin = [1, 1, 0]
#quad = {(0, 1):2, (1, 2):-1}
#bqm = dimod.BinaryQuadraticModel(lin, quad)

In [ ]:
#bqm.to_ising()

In [ ]:
#bqm = dimod.BinaryQuadraticModel({'a': 0.0, 'b': -1.0, 'c': 0.5},　{('a', 'b'): -1.0, ('b', 'c'): 1.5},　1.4,　dimod.SPIN)
#bqm.to_qubo()

## dimod.BinaryQuadraticModel

In [ ]:
bqm = dimod.BinaryQuadraticModel({0: -1, 1: -1}, {(0, 1): 2}, 0.0, dimod.BINARY)
bqm_ising = bqm.change_vartype(dimod.SPIN, inplace=False)
bqm_ising 

In [ ]:
h = {0: 0.0, 1: 0.0}
J = {(0, 1): -1.0}
bqm = dimod.BinaryQuadraticModel.from_ising(h, J)
response = dimod.ExactSolver().sample(bqm)
for sample, energy in response.data(['sample', 'energy']): 
    print(sample, energy)

In [ ]:
h = {'a': -0.5, 'b': 1.0}
J = {('a', 'b'): -1.5}
sampleset = dimod.ExactSolver().sample_ising(h, J)
print(sampleset)

In [ ]:
Q = {('a', 'b'): 2.0, ('a', 'a'): 1.0, ('b', 'b'): -0.5}
sampleset = dimod.ExactSolver().sample_qubo(Q)
sampleset

In [ ]:
bqm = dimod.BinaryQuadraticModel({'a': 1.5}, {('a', 'b'): -1}, 0.0, 'SPIN')
sampleset = dimod.ExactSolver().sample(bqm)
sampleset

In [ ]:
bqm = dimod.BinaryQuadraticModel.from_qubo({('a', 'b'): 1})
sampler = dimod.NullSampler()
sampleset = sampler.sample(bqm)
len(sampleset)

In [ ]:
bqm = dimod.BinaryQuadraticModel.from_qubo({('a', 'b'): 1})
sampler = dimod.NullSampler(parameters=['a'])
sampleset = sampler.sample(bqm, a=5)
sampleset 

In [ ]:
h = {'a': -0.5, 'b': 1.0}
J = {('a', 'b'): -1.5}
sampleset = dimod.SimulatedAnnealingSampler().sample_ising(h, J)
sampleset